# Compute BlocRes Evaluation

In [1]:
import os
from pathlib import Path
import numpy as np

import scipion_bridge
import scipion_bridge.ffi.scipion as scipion
# from scipion_bridge.ffi.blocres import blocres
# from scipion_bridge.proxy import OutputInfo

import shutil

from utils import download as D
scipion_bridge.utils.environment.configure_default_env()

import matplotlib.pyplot as plt

### Download Data

In [2]:
DOWNLOAD_PATH = Path("data/downloaded")
EMDB_ENTRY = 41510

In [3]:
metadata = D.download_emdb_metadata(entry_id=EMDB_ENTRY)

os.makedirs(DOWNLOAD_PATH, exist_ok=True)

emdb_map = D.download_emdb_map(EMDB_ENTRY, DOWNLOAD_PATH)
map_1, map_2 = D.download_halfmaps(EMDB_ENTRY, DOWNLOAD_PATH)
pdb_model = D.download_pdb_model("8tqo", DOWNLOAD_PATH) # metadata.pdb_id

metadata

EMDBMetadata(pdb_id='7l70', resolution=3.1, sampling=0.835, size=256, org_x=0, org_y=0, org_z=0)

### Create Volume from PDB

In [17]:
volume = scipion.xmipp_volume_from_pdb(
    pdb_model,
    center_pdb="-v 0",
    sampling=metadata.sampling,
    size=metadata.size,
).typed(astype=scipion_bridge.typed.volume.SpiderFile, copy_data=False)

volume = scipion.xmipp_volume_align(
    embdb_map=emdb_map,
    volume=volume,
    local=True,
)

volume

[XMIPP] scipion run xmipp_volume_from_pdb -i data/downloaded/pdb8tqo.ent -o /tmp/tmp_2969cjl --centerPDB -v 0 --sampling 0.835 --size 256
Xmipp command detected
Scipion v3.7.1 - Eugenius
[XMIPP] scipion run xmipp_volume_align --i1 data/downloaded/emd_41510.map --i2 /tmp/tmp_2969cjl.vol --local --apply /tmp/tmpjqimc0r_.vol
1 (1,0,0,0,0,1,0,0,0)--->-0.349795
   (1,0,***0.709587,0,0,1,0,0,0)--->-0.354899
   (1,0,0.709587,***0.889898,0,1,0,0,0)--->-0.359015
   (1,0,0.709587,0.889898,***-2.02647,1,0,0,0)--->-0.364126
   (1,0,0.709587,0.889898,-2.02647,***1.0526,0,0,0)--->-0.372631
   (1,0,0.709587,0.889898,-2.02647,1.0526,***-4.86482,0,0)--->-0.388258
   (1,0,0.709587,0.889898,-2.02647,1.0526,-4.86482,***3.64532,0)--->-0.465488
   (1,0,0.709587,0.889898,-2.02647,1.0526,-4.86482,3.64532,***2.76693)--->-0.558715
2 (1,0,0.709587,0.889898,-2.02647,1.0526,-4.86482,3.64532,2.76693)--->-0.558715
   (1,0,***2.07012,0.889898,-2.02647,1.0526,-4.86482,3.64532,2.76693)--->-0.576119
   (1,0,2.07012,***0

Output()

In [6]:
from ipywidgets import interact, IntSlider
@interact(i=IntSlider(min=0, max=255))
def plot_slices(i=125):
    plt.imshow(np.array(volume)[i], cmap="gray")

interactive(children=(IntSlider(value=0, description='i', max=255), Output()), _dom_classes=('widget-interact'…

In [6]:
# [XMIPP] scipion run xmipp_volume_align --i1 data/downloaded/emd_41510.map --i2 /tmp/tmpvmzz70hb --local --apply /tmp/tmp09zqxngg.vol

In [7]:
volume

### Mask from Volume

In [7]:
import logging


mask = scipion.xmipp_transform_threshold(
    volume, select="below 0.02", substitute="binarize"
)

mask

[XMIPP] scipion run xmipp_transform_threshold -i /tmp/tmpkzz5xy7e.vol -o /tmp/tmpfw9sjx3z.vol --select below 0.02 --substitute binarize
Input File: /tmp/tmpkzz5xy7e.vol
Output File: /tmp/tmpfw9sjx3z.vol
Xmipp command detected
Scipion v3.7.1 - Eugenius


Output()

In [16]:
np_array = np.random.normal(-1, 1, [256, 256, 256])
scipion.xmipp_transform_threshold(
    np_array, select="above 0.5", substitute="binarize"
)

[XMIPP] scipion run xmipp_transform_threshold -i /tmp/tmpdg8wswwh.vol -o /tmp/tmpv1hqd8_1.vol --select above 0.5 --substitute binarize
Input File: /tmp/tmpdg8wswwh.vol
Output File: /tmp/tmpv1hqd8_1.vol
Xmipp command detected
Scipion v3.7.1 - Eugenius


Output()

In [14]:
!scipion run xmipp_transform_threshold

Scipion v3.7.1 - Eugenius
Xmipp command detected
This method will return the plugin class in the future. Please use getPluginModule instead
PROGRAM
   xmipp_transform_threshold
USAGE
   Threshold volumes and images 
SEE ALSO
   transform_mask, transform_morphology
OPTIONS

   -i, --input <input_file>
          Input file: metadata, stack, volume or image. 
   [-o, --output <output_file=>]
          Output file: metadata, stack, volume or image. 
   [--oroot <root=>]
          Rootname of output individual images. 
          Output image format can be set adding extension after rootname as ":ext". 
   --select <mode>
           Select pixels meeting 
      where <mode> can be:
        abs_below <th>
           Absolute value below a threshold 
        below <th>
           Below a threshold 
        above <th>
           Above a threshold 
   --substitute <substitutionMode=value>
           Substitute selected pixels by 
      where <substitutionMode> can be:
        binarize
          

In [ ]:
def apply_sampling(data, sampling):
    data = scipion.xmipp_image_convert(data)
    scipion.xmipp_image_header(data, sampling=sampling)

    return data

volume = apply_sampling(volume, metadata.sampling)
mask = apply_sampling(mask, metadata.sampling)

### Compute BlocRes

In [7]:
blocres_half = blocres(
    str(map_1),
    str(map_2),
    mask=mask,
    sampling=f"{metadata.sampling} {metadata.sampling} {metadata.sampling}",
    box=int(metadata.resolution * 6),
    cutoff=0.5,
    step=1,
    verbose=1,
)

[BlocRes] /home/max/scipion3/software/em/bsoft/bin/blocres  -Mask /tmp/tmpjvct5xl5.mrc -sampling 0.835 0.835 0.835 -box 18 -cutoff 0.5 -nofill -smooth -pad 1 -step 1 -maxresolution 0.5 -verbose 1 data/downloaded/emd_41510_half_map_1.map data/downloaded/emd_41510_half_map_2.map /tmp/tmpunxxt95c.map
# /home/max/scipion3/software/em/bsoft/bin/blocres -Mask /tmp/tmpjvct5xl5.mrc -sampling 0.835 0.835 0.835 -box 18 -cutoff 0.5 -nofill -smooth -pad 1 -step 1 -maxresolution 0.5 -verbose 1 data/downloaded/emd_41510_half_map_1.map data/downloaded/emd_41510_half_map_2.map /tmp/tmpunxxt95c.map 
# Wed Jul 16 13:15:13 2025


Calculating local resolution:
Mask:                           /tmp/tmpjvct5xl5.mrc (-1)
Kernel size:                    18
Step size:                      1
Edge size:                      9	9	9
Padding size:                   36
Smoothing/tapering:             2
Cutoff(s):                      0.500

Boxes to calculate:             646612

Boxes calculated:               646612

In [8]:
atomic_model = scipion.xmipp_pdb_label_from_volume(
    pdb=str(pdb_model),
    volume=blocres_half,
    mask=mask,
    sampling=metadata.sampling,
    origin="%f %f %f" % (metadata.org_x, metadata.org_y, metadata.org_z),
)

shutil.copy(atomic_model.path, f"data/blocres_result_{EMDB_ENTRY}.pdb")

[XMIPP] scipion run xmipp_pdb_label_from_volume -o /tmp/tmp7efz7quw.atom.pdb --pdb data/downloaded/pdb8tqo.ent --vol /tmp/tmpcwzph5sb.map --mask /tmp/tmpcub3ywqh.mrc --sampling 0.835 --origin 0.000000 0.000000 0.000000
PDB file:           data/downloaded/pdb8tqo.ent
Output:       /tmp/tmp7efz7quw.atom.pdb
Sampling rate:        0.835
Origin:               0.000000 0.000000 0.000000 
Radius:               0.8
mean value: = 3.2293
absolute mean value: = 3.2293
Xmipp command detected
Scipion v3.7.1 - Eugenius


'data/blocres_result_41510.pdb'